<a href="https://colab.research.google.com/github/pavel7479/test_task1/blob/main/task01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task1

## Описание задачи:
Написать программу, которая распознает голосовые фразы:
* "привет я разработчик" и отвечает "сегодня выходной".
* "Я сегодня не приду домой" и отвечает "Ну и катись отсюда".
## Требования:
* Провести анализ и сравнение моделей для распознавания текста.
* Выбрать лучшую модель с учетом скорости распознавания и обосновать свой выбор.
### Для генерации текста (проговаривания ответа) можно использовать любую модель.
## Ожидаемые результаты:
* Программа должна корректно распознавать указанные голосовые фразы и давать соответствующий ответ.
* Подробный анализ моделей для распознавания текста, включая сравнение точности и скорости выполнения.
* Обоснование выбора модели.


In [1]:
# @title Установка библиотек

!pip install SpeechRecognition
!pip install transformers
!pip install pyttsx3
!pip install vosk
!pip install soundfile
!apt-get install -y espeak
!pip install pydub
!sudo apt-get install ffmpeg
!pip install pydub
!pip install SpeechRecognition
!apt-get install -y ffmpeg



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 20.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 9.8 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=af52d6521a317160a360ef031a9a1c275299504f8756c293eefd804c50d43cd1
  Stored in directory: /root/.cache/pip/wheels/d7/31/a1/18e1e7e8bfdafd19e6803d7eb919b563dd11de380e4304e332
Successfully built srt
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  espeak-data libespeak1 libportaudio2 libsonic0
The following NEW packages will be installed:
  espeak espeak-data libespeak1 libportaudio2 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 1,382 kB of archives.
After this operation, 3,178 kB of additional

In [2]:
# @title Импорт модулей

import speech_recognition as sr
import pyttsx3
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import torch
from vosk import Model, KaldiRecognizer
import wave
import time
import os
import time
import wave
import pyttsx3
import speech_recognition as sr
from pydub import AudioSegment
from scipy.io.wavfile import write
import soundfile as sf
from IPython.display import Audio
import requests
from IPython.display import Audio, display


In [ ]:
# @title Распознование с помощью SpeechRecognition

# Функция для распознавания речи
def recognize_speech(audio_path):
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio_data = recognizer.record(source)  # Используем метод record для чтения данных из файла
        try:
            text = recognizer.recognize_google(audio_data, language="ru-RU")
            return text.lower()  # Возвращаем текст в нижнем регистре
        except sr.UnknownValueError:
            return "Не удалось распознать речь"
        except sr.RequestError:
            return "Ошибка сервиса распознавания"

# Функция для конвертации аудиофайла
def convert_audio_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(output_file, format="wav")

# Функция для синтеза речи и сохранения в файл
def speak(text, filename):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    russian_voice_found = False
    for voice in voices:
        if "russian" in voice.name.lower():
            engine.setProperty('voice', voice.id)
            russian_voice_found = True
            break
    if not russian_voice_found:
        print("Русский голос не найден. Используется голос по умолчанию.")
    engine.save_to_file(text, filename)  # Сохраняем речь в файл
    engine.runAndWait()

# Функция для синтеза речи и сохранения в файл
def main(audio_path):
    start_time = time.time()  # Запуск таймера времени выполнения

    # Конвертация аудиофайла в нужный формат
    converted_audio_path = "/content/converted_audio.wav"
    convert_audio_to_wav(audio_path, converted_audio_path)

    # Распознавание речи
    recognized_text = recognize_speech(converted_audio_path)
    print("Распознанный текст:", recognized_text)

    # Нормализация текста
    normalized_text = recognized_text.lower().strip()
    print("Нормализованный текст:", normalized_text)

    # Ответ на основе распознанного текста
    if "привет я разработчик" in normalized_text:
        response = "сегодня выходной"
    elif "я сегодня не приду домой" in normalized_text:
        response = "ну и катись отсюда"
    else:
        response = "Я не понимаю эту фразу."

    # Синтез речи и сохранение в файл с уникальным именем
    response_audio_path = f'/content/response_{int(time.time())}.wav'  # Используем текущее время в секундах для уникального имени файла
    speak(response, response_audio_path)

    time.sleep(1)
    # Воспроизведение ответа
    display(Audio(response_audio_path, rate=16000, autoplay=True))

    # Время выполнения
    elapsed_time = time.time() - start_time
    print(f"Время выполнения: {elapsed_time:.2f} секунд")

# Загрузка аудиофайла
from google.colab import files
uploaded = files.upload()

# Использование загруженного файла
for filename in uploaded.keys():
    audio_path = filename
    main(audio_path)


Saving 1.mp3 to 1 (2).mp3
Распознанный текст: привет я разработчик
Нормализованный текст: привет я разработчик


Exception ignored on calling ctypes callback function: <bound method EspeakDriver._onSynth of <pyttsx3.drivers.espeak.EspeakDriver object at 0x7fd30a2d9090>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyttsx3/drivers/espeak.py", line 171, in _onSynth
    self._proxy.notify('finished-utterance', completed=True)
ReferenceError: weakly-referenced object no longer exists


Время выполнения: 1.79 секунд


In [ ]:
# @title Распознование с помощью Vosk
!pip install vosk

In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip


--2024-07-13 14:00:37--  https://alphacephei.com/vosk/models/vosk-model-small-ru-0.22.zip
Resolving alphacephei.com (alphacephei.com)... 188.40.21.16, 2a01:4f8:13a:279f::2
Connecting to alphacephei.com (alphacephei.com)|188.40.21.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46236750 (44M) [application/zip]
Saving to: ‘vosk-model-small-ru-0.22.zip’

vosk-model-small-ru 100%[===================>]  44.09M  20.0MB/s    in 2.2s    

2024-07-13 14:00:40 (20.0 MB/s) - ‘vosk-model-small-ru-0.22.zip’ saved [46236750/46236750]



In [ ]:
!unzip vosk-model-small-ru-0.22.zip

Archive:  vosk-model-small-ru-0.22.zip
   creating: vosk-model-small-ru-0.22/
   creating: vosk-model-small-ru-0.22/graph/
   creating: vosk-model-small-ru-0.22/graph/phones/
  inflating: vosk-model-small-ru-0.22/graph/phones/word_boundary.int  
  inflating: vosk-model-small-ru-0.22/graph/Gr.fst  
  inflating: vosk-model-small-ru-0.22/graph/HCLr.fst  
  inflating: vosk-model-small-ru-0.22/graph/disambig_tid.int  
   creating: vosk-model-small-ru-0.22/am/
  inflating: vosk-model-small-ru-0.22/am/final.mdl  
  inflating: vosk-model-small-ru-0.22/README  
   creating: vosk-model-small-ru-0.22/conf/
  inflating: vosk-model-small-ru-0.22/conf/model.conf  
  inflating: vosk-model-small-ru-0.22/conf/mfcc.conf  
   creating: vosk-model-small-ru-0.22/ivector/
  inflating: vosk-model-small-ru-0.22/ivector/final.dubm  
  inflating: vosk-model-small-ru-0.22/ivector/global_cmvn.stats  
  inflating: vosk-model-small-ru-0.22/ivector/final.ie  
  inflating: vosk-model-small-ru-0.22/ivector/final.mat  

In [ ]:
import vosk
import wave
import json
import pyttsx3
import time
from google.colab import files

model_path = "/content/vosk-model-small-ru-0.22"
# Функция для конвертации аудиофайла в формат моно PCM с частотой 16000 Hz
def convert_audio_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(output_file, format="wav")

# Функция для распознавания речи с использованием Vosk
def recognize_speech_vosk(audio_path):
    model = vosk.Model(model_path)  # Укажите путь к вашей модели
    wf = wave.open(audio_path, "rb")
    recognizer = vosk.KaldiRecognizer(model, wf.getframerate())

    recognized_text = ""
    while True:
        data = wf.readframes(4000)
        if len(data) == 0:
            break
        if recognizer.AcceptWaveform(data):
            result = recognizer.Result()
            recognized_text += json.loads(result)['text'] + " "

    recognized_text += json.loads(recognizer.FinalResult())['text']
    return recognized_text.strip().lower()

# Функция для синтеза речи и сохранения в файл
def speak(text, filename):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    russian_voice_found = False
    for voice in voices:
        if "russian" in voice.name.lower():
            engine.setProperty('voice', voice.id)
            russian_voice_found = True
            break
    if not russian_voice_found:
        print("Русский голос не найден. Используется голос по умолчанию.")
    engine.save_to_file(text, filename)
    engine.runAndWait()

# Функция для вывода ответа
def display_response(response):
    print("Ответ:", response)

# Основная функция
def main(audio_path):
    start_time = time.time()

    converted_audio_path = "/content/converted_audio.wav"
    convert_audio_to_wav(audio_path, converted_audio_path)

    recognized_text = recognize_speech_vosk(converted_audio_path)
    print("Распознанный текст:", recognized_text)

    normalized_text = recognized_text.strip().lower()

    if not normalized_text:
        print("Не удалось распознать речь.")
        return

    if "привет я разработчик" in normalized_text:
        response = "сегодня выходной"
    elif "я сегодня не приду домой" in normalized_text:
        response = "ну и катись отсюда"
    else:
        response = "Я не понимаю эту фразу."

    display_response(response)  # Выводим ответ в текстовом виде

    response_audio_path = "/content/response.wav"
    speak(response, response_audio_path)  # Синтезируем ответ

    time.sleep(1)
    # Воспроизводим ответ
    display(Audio(response_audio_path, rate=16000, autoplay=True))

    elapsed_time = time.time() - start_time
    print(f"Время выполнения: {elapsed_time:.2f} секунд")

# Загрузка аудиофайла
from google.colab import files
uploaded = files.upload()

# Использование загруженного файла
for filename in uploaded.keys():
    audio_path = filename
    main(audio_path)


Saving 3.mp3 to 3 (2).mp3
Распознанный текст: совсем другой разговор
Ответ: Я не понимаю эту фразу.


Exception ignored on calling ctypes callback function: <bound method EspeakDriver._onSynth of <pyttsx3.drivers.espeak.EspeakDriver object at 0x7fd309469bd0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyttsx3/drivers/espeak.py", line 171, in _onSynth
    self._proxy.notify('finished-utterance', completed=True)
ReferenceError: weakly-referenced object no longer exists


Время выполнения: 3.19 секунд


In [9]:
!pip install jsonlines

In [39]:
# @title Распознование с помощью Wit.ai
import json
import jsonlines
import re

wit_api_token = '5ZHYCOJPB3OUKSZPIEOC4UQOITLAOUH7'

# Функция для конвертации аудиофайла в формат моно PCM с частотой 16000 Hz
def convert_audio_to_wav(input_file, output_file):
    audio = AudioSegment.from_file(input_file)
    audio = audio.set_frame_rate(16000).set_channels(1).set_sample_width(2)
    audio.export(output_file, format="wav")

# Функция для распознавания речи с использованием Wit.ai
def recognize_speech_wit(audio_path):
    with open(audio_path, 'rb') as audio_file:
        response = requests.post('https://api.wit.ai/speech', headers={'Authorization': f'Bearer {wit_api_token}', 'Content-Type': 'audio/wav'}, data=audio_file)

        # Разбиваем текст по строкам
        lines = response.text.splitlines()
        fourth_from_end = lines[-4]

        match = re.search(r'"text": "(.*?)"', fourth_from_end)

        if match:
            extracted_text = match.group(1)  # Извлекаем текст
            print(extracted_text)
        else:
            print("Текст не найден.")

        return extracted_text.strip().lower()

# Функция для синтеза речи и сохранения в файл
def speak(text, filename):
    engine = pyttsx3.init()
    voices = engine.getProperty('voices')
    russian_voice_found = False
    for voice in voices:
        if "russian" in voice.name.lower():
            engine.setProperty('voice', voice.id)
            russian_voice_found = True
            break
    if not russian_voice_found:
        print("Русский голос не найден. Используется голос по умолчанию.")
    engine.save_to_file(text, filename)
    engine.runAndWait()

# Функция для вывода ответа
def display_response(response):
    print("Ответ:", response)

# Функция генерации ответа на основе распознанного текста
def generate_response(recognized_text):
    normalized_text = recognized_text.strip().lower()
    if "привет, я разработчик" in normalized_text:
        return "сегодня выходной"
    elif "я сегодня не приду домой" in normalized_text:
        return "ну и катись отсюда"
    else:
        return "Я не понимаю эту фразу."

# Основная функция
def main(audio_path):
    start_time = time.time()

    converted_audio_path = "converted_audio.wav"
    convert_audio_to_wav(audio_path, converted_audio_path)

    recognized_text = recognize_speech_wit(converted_audio_path)
    print("Распознанный текст:", recognized_text)

    if recognized_text is None:
        print("Не удалось распознать речь.")
        return

    response = generate_response(recognized_text)

    display_response(response)  # Выводим ответ в текстовом виде

    response_audio_path = "response.wav"
    speak(response, response_audio_path)  # Синтезируем ответ

    # Воспроизводим ответ
    time.sleep(1)  # Небольшая задержка для корректного воспроизведения
    display(Audio(response_audio_path, rate=16000, autoplay=True))

    elapsed_time = time.time() - start_time
    print(f"Время выполнения: {elapsed_time:.2f} секунд")

# Загрузка аудиофайла
from google.colab import files
uploaded = files.upload()

# Использование загруженного файла
for filename in uploaded.keys():
    audio_path = filename
    main(audio_path)



Saving 1.mp3 to 1 (31).mp3
Привет, я разработчик
Распознанный текст: привет, я разработчик
Ответ: сегодня выходной


Exception ignored on calling ctypes callback function: <bound method EspeakDriver._onSynth of <pyttsx3.drivers.espeak.EspeakDriver object at 0x7f8ec554f700>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pyttsx3/drivers/espeak.py", line 171, in _onSynth
    self._proxy.notify('finished-utterance', completed=True)
ReferenceError: weakly-referenced object no longer exists


Время выполнения: 3.62 секунд
